In [1]:
import numpy as np
import scipy 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random
import os
import pickle
import glob 
import itertools
import json
from scipy.stats import ttest_rel
from collections import Counter
import statsmodels.api as sm
from patsy import dmatrices
from matplotlib.collections import LineCollection
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import sklearn
import json
import pickle
import arviz as az
import pingouin

import matplotlib.gridspec as gridspec
from matplotlib.patches import FancyBboxPatch

import statsmodels.formula.api as smf
import pandas as pd

import jax
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.pipeline import make_pipeline
import re

plt.rcParams['font.family'] = "Arial"
# --- Set the desired confidence interval probability ---
az.rcParams["stats.ci_prob"] = 0.95

figpath = os.path.join("fig")
fitspath = os.path.join("modelfits")
data_savepath = os.path.join("data")
# label_path = os.path.join("..","experiment","img_Ntrain1050_largenoise", "stimuli.csv")

# Visualization and Efron term computational setup
experiment_name_temps = ["exps","exps","exps","exps"]
experiment_group_temps = ["exp1","exp2","exp3","exp4"]
use_Vtrue_inEfron = True

### Sanity check: Efron's testMSE_estimate

In [2]:
# Compare Efron's estimated E[testMSE] with empirical E[testMSE]

N_trains = [10,50]
    
exp_count = 0
for exp_idx in range(len(experiment_name_temps)):
    experiment_name_temp = experiment_name_temps[exp_idx]
    experiment_group_temp = experiment_group_temps[exp_idx]
    print(experiment_group_temp)

    if(experiment_group_temp=="exp1"): # small noise dataset
        model_names = ["linear","five_poly"]
        mse_cols = [model_name+"_MSE_Efron" for model_name in model_names]
        model_dfs = [2,6]
        model_possible_label_idxs = (0,1)
        noise_sd = 0.08
        firstcond_colname_temp = "first_condition_isfivepoly"
        data_responseispoly_colname_temp = "response_isfivepoly"
        label_path_temp = os.path.join("..","experiment","img_Ntrain1050", "stimuli.csv")
        
    else: # Large noise dataset
        model_names = ["linear","five_poly","eight_poly"]
        mse_cols = [model_name+"_MSE_Efron" for model_name in model_names]
        model_dfs = [2,6,9]
        noise_sd = 0.25
        label_path_temp = os.path.join("..","experiment","img_Ntrain1050_largenoise", "stimuli.csv")
        # Which two model families are fitted and compared?
        if(experiment_group_temp=="exp2"):
            model_possible_label_idxs = (0,1)
            firstcond_colname_temp = "first_condition_isfivepoly"
            data_responseispoly_colname_temp = "response_isfivepoly"
        elif(experiment_group_temp=="exp3"):
            model_possible_label_idxs = (0,2)
            firstcond_colname_temp = "first_condition_iseightpoly"
            data_responseispoly_colname_temp = "response_iseightpoly"
        elif(experiment_group_temp=="exp4"):
            model_possible_label_idxs = (1,2)
            firstcond_colname_temp = "first_condition_iseightpoly"
            data_responseispoly_colname_temp = "response_iseightpoly"
        else:
            print("Warning: experiment name or group is wrong!")
            
    # Pair of candidate models used for this dataset
    model_possible_labels = [model_names[model_possible_label_idxs[0]], model_names[model_possible_label_idxs[1]]]

    # Load df containing data labels (true generative model, and fitted model that minimizes test MSE.)
    df_labels_Efron_est = pd.read_csv(label_path_temp, index_col=False) 
    # Stimulus_number -- matching PNG names
    df_labels_Efron_est = df_labels_Efron_est.rename(columns={'Unnamed: 0': 'stimulus_number'})
    
    # Label 0 if should choose simpler model, Label 1 otherwise.
    df_labels_Efron_est["experiment"] = experiment_group_temp
    df_labels_Efron_est["noise_sd"] = noise_sd
    df_labels_Efron_est["ground_truth_labels"] = 1-(df_labels_Efron_est["ground_truth"]==model_possible_labels[0]).astype(int)
    df_labels_Efron_est["MAE_bestmodel_labels"] = (df_labels_Efron_est[model_possible_labels[0]+"_MAE"]>df_labels_Efron_est[model_possible_labels[1]+"_MAE"]).astype(int)
    df_labels_Efron_est["TSE_bestmodel_labels"] = (df_labels_Efron_est[model_possible_labels[0]+"_TSE"]>df_labels_Efron_est[model_possible_labels[1]+"_TSE"]).astype(int)

    # Always simple_model - complex_model
    df_labels_Efron_est["models_trainMSE_diff"] = df_labels_Efron_est[model_possible_labels[0]+"_MSE_train"] -  df_labels_Efron_est[model_possible_labels[1]+"_MSE_train"]
    
    df_labels_Efron_est["MSE_bestmodel_labels_train"] = (df_labels_Efron_est[model_possible_labels[0]+"_MSE_train"]>df_labels_Efron_est[model_possible_labels[1]+"_MSE_train"]).astype(int)
    df_labels_Efron_est["MSE_bestmodel_labels"] = (df_labels_Efron_est[model_possible_labels[0]+"_MSE"]>df_labels_Efron_est[model_possible_labels[1]+"_MSE"]).astype(int)

    # Don't consider the model that was not displayed to subjects in this experiment_group_temp, even if it is the best predictive model
    df_labels_Efron_est["best_model_MAE"] = np.where(df_labels_Efron_est["MAE_bestmodel_labels"], model_possible_labels[1], model_possible_labels[0])
    df_labels_Efron_est["best_model_MSE"] = np.where(df_labels_Efron_est["MSE_bestmodel_labels"], model_possible_labels[1], model_possible_labels[0])
    df_labels_Efron_est["best_model_TSE"] = np.where(df_labels_Efron_est["TSE_bestmodel_labels"], model_possible_labels[1], model_possible_labels[0])
    print("Agreement between MAE- and MSE-minimizing labels:",np.mean(df_labels_Efron_est["best_model_MAE"]==df_labels_Efron_est["best_model_MSE"]))
    print("Agreement between TSE- and MSE-minimizing labels:",np.mean(df_labels_Efron_est["best_model_TSE"]==df_labels_Efron_est["best_model_MSE"]))

    df_labels_Efron_est["experiment"] = experiment_group_temp
    # Compute Efron estimates of E[testMSE] for each model.
    for model_idx in range(len(model_names)):
        model = model_names[model_idx]
        model_df = model_dfs[model_idx]
        if(use_Vtrue_inEfron):
            correction_Efron = 2*(noise_sd**2)*model_df/df_labels_Efron_est["sample_size"]
        else:
            # Use noise_sd = SSE/(N-D), so 2VD/N_train = 2*trainMSE*N*D / ((N-D)*N) = 2*trainMSE*D / (N-D)
            correction_Efron = (2*df_labels_Efron_est[model+"_MSE_train"]*model_df)/(df_labels_Efron_est["sample_size"]-model_df)
        testMSE_Efron = df_labels_Efron_est[model+"_MSE_train"] + correction_Efron
        testMSE_empirical = df_labels_Efron_est[model+"_MSE"]
        df_labels_Efron_est[model+"_MSE_Efron"] = testMSE_Efron
        df_labels_Efron_est[model+"_correction_Efron"] = correction_Efron
        df_labels_Efron_est[model+"_OLSnoise_est"] = (df_labels_Efron_est[model+"_MSE_train"]*df_labels_Efron_est["sample_size"])/(df_labels_Efron_est["sample_size"]-model_df)
        
    mse_cols_relevant = [mse_cols[model_possible_label_idxs[0]], mse_cols[model_possible_label_idxs[1]]]
    df_labels_Efron_est["best_model_MSE_Efron"] = df_labels_Efron_est[mse_cols_relevant].idxmin(axis=1)
    df_labels_Efron_est["best_model_MSE_Efron"] = np.where(df_labels_Efron_est["best_model_MSE_Efron"]==model_possible_labels[1]+"_MSE_Efron", model_possible_labels[1],model_possible_labels[0])
    df_labels_Efron_est["MSE_bestmodel_labels_Efron"] = (df_labels_Efron_est[model_possible_labels[0]+"_MSE_Efron"]>df_labels_Efron_est[model_possible_labels[1]+"_MSE_Efron"]).astype(int)
    
    # Always simple_model - complex_model
    df_labels_Efron_est["models_correctionEfron_diff"] = df_labels_Efron_est[model_possible_labels[0]+"_correction_Efron"] - df_labels_Efron_est[model_possible_labels[1]+"_correction_Efron"] 

    
    # Check confusion matrix: does Efron vs. empirical testMSE predict different choice labels?
    print("Agreement between MAE- and MSEEfron-minimizing labels:",np.mean(df_labels_Efron_est["MAE_bestmodel_labels"]==df_labels_Efron_est["MSE_bestmodel_labels_Efron"]))
    print("Agreement between TSE- and MSEEfron-minimizing labels:",np.mean(df_labels_Efron_est["TSE_bestmodel_labels"]==df_labels_Efron_est["MSE_bestmodel_labels_Efron"]))
    print("Agreement between true and Efron MSE-minimizing labels:",np.mean(df_labels_Efron_est["MSE_bestmodel_labels"]==df_labels_Efron_est["MSE_bestmodel_labels_Efron"]))
    print("N_train=10:",np.mean(df_labels_Efron_est[df_labels_Efron_est["sample_size"]==10]["MSE_bestmodel_labels"]==df_labels_Efron_est[df_labels_Efron_est["sample_size"]==10]["MSE_bestmodel_labels_Efron"]))
    print("N_train=50:",np.mean(df_labels_Efron_est[df_labels_Efron_est["sample_size"]==50]["MSE_bestmodel_labels"]==df_labels_Efron_est[df_labels_Efron_est["sample_size"]==50]["MSE_bestmodel_labels_Efron"]))
    print()
    
    #print(sklearn.metrics.confusion_matrix(df_labels_Efron_est["MSE_bestmodel_labels"] , df_labels_Efron_est["MSE_bestmodel_labels_Efron"]))
    
    # Each variable below is [#trials simple model minimizes trainMSE, #trials complex model minimizes trainMSE].
    Ntrain10_trainMSElabelcounts = df_labels_Efron_est[df_labels_Efron_est["sample_size"]==N_trains[0]]["MSE_bestmodel_labels_train"].value_counts(sort=False).reindex([0,1], fill_value=0).sort_index().to_numpy()
    Ntrain50_trainMSElabelcounts = df_labels_Efron_est[df_labels_Efron_est["sample_size"]==N_trains[1]]["MSE_bestmodel_labels_train"].value_counts(sort=False).reindex([0,1], fill_value=0).sort_index().to_numpy()
    # Each variable below is [#trials simple model minimizes testMSE, #trials complex model minimizes testMSE], where testMSE is defined either w.r.t. empirical or Efron's estimate.
    Ntrain10_emplabelcounts = df_labels_Efron_est[df_labels_Efron_est["sample_size"]==N_trains[0]]["MSE_bestmodel_labels"].value_counts(sort=False).reindex([0,1], fill_value=0).sort_index().to_numpy()
    Ntrain50_emplabelcounts = df_labels_Efron_est[df_labels_Efron_est["sample_size"]==N_trains[1]]["MSE_bestmodel_labels"].value_counts(sort=False).reindex([0,1], fill_value=0).sort_index().to_numpy()
    Ntrain10_Efronlabelcounts = df_labels_Efron_est[df_labels_Efron_est["sample_size"]==N_trains[0]]["MSE_bestmodel_labels_Efron"].value_counts(sort=False).reindex([0,1], fill_value=0).sort_index().to_numpy()
    Ntrain50_Efronlabelcounts = df_labels_Efron_est[df_labels_Efron_est["sample_size"]==N_trains[1]]["MSE_bestmodel_labels_Efron"].value_counts(sort=False).reindex([0,1], fill_value=0).sort_index().to_numpy()
    print("N_train = 10, label counts:", Ntrain10_trainMSElabelcounts, Ntrain10_emplabelcounts, Ntrain10_Efronlabelcounts)
    print("N_train = 50, label counts:", Ntrain50_trainMSElabelcounts, Ntrain50_emplabelcounts, Ntrain50_Efronlabelcounts)

    if(exp_count==0):
        df_allexps = df_labels_Efron_est
    else:
        df_allexps = pd.concat([df_allexps, df_labels_Efron_est])
    exp_count = exp_count+1
    print()
    
df_allexps.to_csv(os.path.join(data_savepath,"df_allexps.csv"),index=False);

exp1
Agreement between MAE- and MSE-minimizing labels: 1.0
Agreement between TSE- and MSE-minimizing labels: 0.875
Agreement between MAE- and MSEEfron-minimizing labels: 0.78125
Agreement between TSE- and MSEEfron-minimizing labels: 0.78125
Agreement between true and Efron MSE-minimizing labels: 0.78125
N_train=10: 0.6666666666666666
N_train=50: 0.8958333333333334

N_train = 10, label counts: [ 0 48] [34 14] [34 14]
N_train = 50, label counts: [ 0 48] [24 24] [23 25]

exp2
Agreement between MAE- and MSE-minimizing labels: 0.9791666666666666
Agreement between TSE- and MSE-minimizing labels: 0.9270833333333334
Agreement between MAE- and MSEEfron-minimizing labels: 0.6875
Agreement between TSE- and MSEEfron-minimizing labels: 0.6145833333333334
Agreement between true and Efron MSE-minimizing labels: 0.6875
N_train=10: 0.7708333333333334
N_train=50: 0.6041666666666666

N_train = 10, label counts: [ 0 48] [42  6] [43  5]
N_train = 50, label counts: [ 0 48] [38 10] [39  9]

exp3
Agreement be

## Load human data (compatible with all experiments)

In [3]:
# Parse datafiles
exp_count = 0
genders_exps = []
ages_exps = []
genders_exps_included = []
ages_exps_included = []
for exp_idx in range(len(experiment_name_temps)):
    genders = []
    ages = []
    genders_included = []
    ages_included = []
    experiment_name_temp = experiment_name_temps[exp_idx]
    experiment_group_temp = experiment_group_temps[exp_idx]
            
    # Parse datafiles
    datapath = os.path.join("..","experiment","data", experiment_name_temp)
    csv_files = glob.glob(os.path.join(datapath, "*.csv"))
    N_trains = [10,50]
    num_conditions = len(N_trains)
    n_subj = len(csv_files)
    num_trials = 96
    n_trials_per_condition = num_trials//2
    rejection_criterion = 20
    max_bonus = 2.5

    # How many subjects, and which subjects are in this experiment group?
    mturk_IDs = []
    for subj in range(n_subj): 
        mturk_ID = csv_files[subj].split("\\")[-1][:-4]
        # read the csv file 
        df_subj = pd.read_csv(csv_files[subj]) 
        experiment_group_subj = df_subj["expIndex"].iloc[0]
        if(experiment_group_subj!=experiment_group_temp): # Subject not in this experiment group
            continue;
        mturk_IDs.append(mturk_ID)
    n_subj = len(mturk_IDs) # n_subj should count subjects in that particular group
    num_attention_checks_passed = np.zeros((n_subj,))
    for subj in range(n_subj): 
        df_subj = pd.read_csv("..\\experiment\\data\\"+experiment_name_temp+"\\" + mturk_IDs[subj]+".csv")
        genders.append(df_subj["gender"].iloc[-1])
        ages.append(df_subj["age"].iloc[-1])
        
        trials_rows = np.where(0+np.isnan(df_subj["trial_number"].to_numpy())==0)[0]
        df_subj_trials = df_subj.iloc[trials_rows,:]
        attention_check_rows = np.where(df_subj_trials["is_attention_check"]>0.5)[0]
        data_rows = np.where(df_subj_trials["is_attention_check"]<=0.5)[0]
        num_attention_checks_passed[subj] = int(np.sum(df_subj_trials.iloc[attention_check_rows,:]["attention_check_passed"]))
        if(num_attention_checks_passed[subj]==4):
            genders_included.append(df_subj["gender"].iloc[-1])
            ages_included.append(df_subj["age"].iloc[-1])
            
    genders_exps.append(genders)
    ages_exps.append(ages)
    genders_exps_included.append(genders_included)
    ages_exps_included.append(ages_included)
    
    import collections
    frequency = collections.Counter(genders)
    frequency_included = collections.Counter(genders_included)
        
    exp_count = exp_count + 1
    print(experiment_group_temp, n_subj, len(genders_included))
    print(frequency)
    print(frequency_included)



exp1 50 50
Counter({'Male': 27, 'Female': 22, 'Other': 1})
Counter({'Male': 27, 'Female': 22, 'Other': 1})
exp2 50 50
Counter({'Male': 27, 'Female': 23})
Counter({'Male': 27, 'Female': 23})
exp3 50 49
Counter({'Female': 27, 'Male': 23})
Counter({'Female': 27, 'Male': 22})
exp4 50 50
Counter({'Male': 30, 'Female': 20})
Counter({'Male': 30, 'Female': 20})


In [4]:
# Parse datafiles
trainMSE_cols = ["linear_MSE_train", "five_poly_MSE_train", "eight_poly_MSE_train"]
trainMAE_cols = ["linear_MAE_train", "five_poly_MAE_train", "eight_poly_MAE_train"]
Efron_correction_cols = ['linear_correction_Efron', 'five_poly_correction_Efron', 'eight_poly_correction_Efron']
Efron_testMSE_cols = ['linear_MSE_Efron', 'five_poly_MSE_Efron', 'eight_poly_MSE_Efron']
testMSE_cols = ['linear_MSE', 'five_poly_MSE', 'eight_poly_MSE']
testMAE_cols = ['linear_MAE', 'five_poly_MAE', 'eight_poly_MAE']
testTSE_cols = ['linear_TSE', 'five_poly_TSE', 'eight_poly_TSE']
OLSnoise_cols = ['linear_OLSnoise_est', 'five_poly_OLSnoise_est', 'eight_poly_OLSnoise_est']
allcols = trainMSE_cols + Efron_correction_cols + Efron_testMSE_cols + testMSE_cols
exp_n_subjs_full = []
exp_count = 0
for exp_idx in range(len(experiment_name_temps)):
    experiment_name_temp = experiment_name_temps[exp_idx]
    experiment_group_temp = experiment_group_temps[exp_idx]

    if(experiment_group_temp=="exp1"): # small noise dataset
        model_names = ["linear","five_poly"]
        mse_cols = [model_name+"_MSE_Efron" for model_name in model_names]
        model_dfs = [2,6]
        model_possible_label_idxs = (0,1)
        noise_sd = 0.08
        firstcond_colname = "first_condition_isfivepoly"
        data_responseispoly_colname = "response_isfivepoly"
        label_path_temp = os.path.join("..","experiment","img_Ntrain1050", "stimuli.csv")
    else: # Large noise dataset
        model_names = ["linear","five_poly","eight_poly"]
        mse_cols = [model_name+"_MSE_Efron" for model_name in model_names]
        model_dfs = [2,6,9]
        noise_sd = 0.25
        label_path_temp = os.path.join("..","experiment","img_Ntrain1050_largenoise", "stimuli.csv")
        if(experiment_group_temp=="exp2"):
            model_possible_label_idxs = (0,1)
            firstcond_colname = "first_condition_isfivepoly"
            data_responseispoly_colname = "response_isfivepoly"
        elif(experiment_group_temp=="exp3"):
            model_possible_label_idxs = (0,2)
            firstcond_colname = "first_condition_iseightpoly"
            data_responseispoly_colname = "response_iseightpoly"
        elif(experiment_group_temp=="exp4"):
            model_possible_label_idxs = (1,2)
            firstcond_colname = "first_condition_iseightpoly"
            data_responseispoly_colname = "response_iseightpoly"
            
        
    # Difference in model complexity: again simple_model - complex_model
    dD = model_dfs[model_possible_label_idxs[0]] - model_dfs[model_possible_label_idxs[1]]
    
    # Parse datafiles
    datapath = os.path.join("..","experiment","data", experiment_name_temp)
    csv_files = glob.glob(os.path.join(datapath, "*.csv"))
    N_trains = [10,50]
    num_conditions = len(N_trains)
    n_subj = len(csv_files)
    num_trials = 96
    n_trials_per_condition = num_trials//2
    rejection_criterion = 20
    max_bonus = 2.5

    # How many subjects, and which subjects are in this experiment group?
    mturk_IDs = []
    for subj in range(n_subj): 
        mturk_ID = csv_files[subj].split("\\")[-1][:-4]
        # read the csv file 
        df_subj = pd.read_csv(csv_files[subj]) 
        experiment_group_subj = df_subj["expIndex"].iloc[0]
        if(experiment_group_subj!=experiment_group_temp): # Subject not in this experiment group
            continue;
        mturk_IDs.append(mturk_ID)
    n_subj = len(mturk_IDs) # n_subj should count subjects in that particular group
    print(n_subj)
    exp_n_subjs_full.append(n_subj)

    completion_time = np.zeros((n_subj))
    first_condition_ispoly = np.zeros((n_subj))
    num_attention_checks_passed = np.zeros((n_subj))
    truegenmodel_bias = np.zeros((num_conditions,n_subj))
    prop_chooseright = np.zeros((num_conditions,n_subj))
    prop_choosepoly = np.zeros((num_conditions,n_subj))
    MAEmodel_bias = np.zeros((num_conditions,n_subj))
    MSEmodel_bias = np.zeros((num_conditions,n_subj))
    accuracy_truegen = np.zeros((num_conditions,n_subj))
    accuracy_MAE = np.zeros((num_conditions,n_subj))
    accuracy_MSE = np.zeros((num_conditions,n_subj))


    for subj in range(n_subj): 
        df_subj = pd.read_csv("..\\experiment\\data\\"+experiment_name_temp+"\\" + mturk_IDs[subj]+".csv")

        trials_rows = np.where(0+np.isnan(df_subj["trial_number"].to_numpy())==0)[0]
        df_subj_trials = df_subj.iloc[trials_rows,:]
        first_condition_ispoly[subj] = df_subj_trials[firstcond_colname].to_numpy()[0]
        attention_check_rows = np.where(df_subj_trials["is_attention_check"]>0.5)[0]
        data_rows = np.where(df_subj_trials["is_attention_check"]<=0.5)[0]
        num_attention_checks_passed[subj] = int(np.sum(df_subj_trials.iloc[attention_check_rows,:]["attention_check_passed"]))

        # Append the correct labels for each trial. 
        df_subj_data = df_subj_trials.iloc[data_rows,:]
        merged_df = df_subj_data.copy() #pd.merge(df_subj_data, df_label, on='stimulus_number')
        merged_df = merged_df.rename(columns={"N_train":"sample_size"})

        # Save df for all trials of all subjects
        merged_df_keeptrialcols = [firstcond_colname, "sample_size","stimulus_number","trial_number","response_raw", data_responseispoly_colname]
        df_subjdata = merged_df[merged_df_keeptrialcols]
        df_subjdata["experiment"] = experiment_group_temp
        df_subjdata["Subject"] = subj
        df_subjdata["mturkID"] = mturk_IDs[subj]
        df_subjdata["#attention_checks_passed"] = num_attention_checks_passed[subj]
        df_subjdata["dD"] = dD


        # Add additional Efron labels and corrected MSE values from df_allexps. Hence the overwrite using suffices. 
        df_subjdata_merged = pd.merge(df_subjdata, df_allexps, on=['experiment', 'sample_size', 'stimulus_number'], how='left', suffixes=("_l",""))

        # OLS noise estimate for a trial: average the OLS noise estimate of the two fitted models visualized.
        df_subjdata_merged["OLSnoise_est_mean"] = (df_subjdata_merged[OLSnoise_cols[model_possible_label_idxs[0]]] + df_subjdata_merged[OLSnoise_cols[model_possible_label_idxs[1]]])/2
        rename_columns = {firstcond_colname:"first_condition_is50", 
              "response_raw": "response_chooseright",
              data_responseispoly_colname: "response_choosecomplexmodel",
              trainMSE_cols[model_possible_label_idxs[0]]: "trainMSE_simplemodel",
              trainMSE_cols[model_possible_label_idxs[1]]: "trainMSE_complexmodel",
              testMSE_cols[model_possible_label_idxs[0]]: "testMSE_simplemodel",
              testMSE_cols[model_possible_label_idxs[1]]: "testMSE_complexmodel",
              Efron_testMSE_cols[model_possible_label_idxs[0]]: "testMSE_Efron_simplemodel",
              Efron_testMSE_cols[model_possible_label_idxs[1]]: "testMSE_Efron_complexmodel",
              Efron_correction_cols[model_possible_label_idxs[0]]: "correction_Efron_simplemodel",
              Efron_correction_cols[model_possible_label_idxs[1]]: "correction_Efron_complexmodel",
              trainMAE_cols[model_possible_label_idxs[0]]: "trainMAE_simplemodel",
              trainMAE_cols[model_possible_label_idxs[1]]: "trainMAE_complexmodel",
              testMAE_cols[model_possible_label_idxs[0]]: "testMAE_simplemodel",
              testMAE_cols[model_possible_label_idxs[1]]: "testMAE_complexmodel",
              testTSE_cols[model_possible_label_idxs[0]]: "testTSE_simplemodel",
              testTSE_cols[model_possible_label_idxs[1]]: "testTSE_complexmodel",
              "MSE_bestmodel_labels": "response_correctiscomplexmodel",}
        df_subjdata_merged = df_subjdata_merged.rename(columns=rename_columns)
        if(exp_count==0 and subj==0):
            df_allsubjs_trials = df_subjdata_merged.copy()
        else:
            if(exp_count==3):
                print(df_subjdata_merged["trainMSE_complexmodel"])
            df_allsubjs_trials = pd.concat([df_allsubjs_trials, df_subjdata_merged])

        #merged_df[data_responseispoly_colname] = merged_df["response_ispoly_Efron"]
        completion_time[subj] = np.max(merged_df["time_elapsed"])/60000 # in minutes

    
    # Save as df of subject_averaged statistics
    df_colnames = ["Subject","first_condition_is50","N_train","#attention_checks_passed"]
    for N_train_idx in range(num_conditions):
        N_train = N_trains[N_train_idx]

        condition_data = np.column_stack((np.array(range(n_subj)).astype(int).flatten(), first_condition_ispoly.astype(int).flatten(), np.repeat(N_train, n_subj).astype(int), num_attention_checks_passed.flatten()))
        if(exp_count==0 and N_train_idx==0):
            df_allsubjs = pd.DataFrame(condition_data, columns=df_colnames)
            df_allsubjs["experiment"] = experiment_group_temp
        else:
            df_cond = pd.DataFrame(condition_data, columns=df_colnames)
            df_cond["experiment"] = experiment_group_temp
            df_allsubjs = pd.concat([df_allsubjs, df_cond])
    df_allsubjs["first_condition"] = df_allsubjs["first_condition_is50"]*(50-10)+10
    #df_allsubjs['N_train'] = pd.Categorical(df_allsubjs['N_train'], categories=np.unique(df_allsubjs['N_train']), ordered=True)
    # Exclude participants who failed any attention check.
    df_allsubjs_included = df_allsubjs[df_allsubjs["#attention_checks_passed"]>=4]
    exp_count = exp_count + 1
    print(experiment_group_temp, n_subj)

# Exclude subjects who failed any attention check.
df_allsubjs_trials_included = df_allsubjs_trials[df_allsubjs_trials["#attention_checks_passed"]>=4]



50


C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subjdata["experiment"] = experiment_group_temp
C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subjdata["Subject"] = subj
C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subjdata["experiment"] = experiment_group_temp
C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subjdata["Subject"] = subj
C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subjdata["experiment"] = experiment_group_temp
C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subjdata["Subject"] = subj
C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subjdata["experiment"] = experiment_group_temp
C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subjdata["Subject"] = subj
C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

...

C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subjdata["experiment"] = experiment_group_temp
C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subjdata["Subject"] = subj
C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

0     0.027377
1     0.000314
2     0.002102
3     0.018143
4     0.008120
        ...   
91    0.064295
92    0.043333
93    0.037266
94    0.047168
95    0.047060
Name: trainMSE_complexmodel, Length: 96, dtype: float64
0     0.053130
1     0.057632
2     0.066143
3     0.050326
4     0.043333
        ...   
91    0.004164
92    0.000063
93    0.002837
94    0.000010
95    0.008007
Name: trainMSE_complexmodel, Length: 96, dtype: float64
0     0.050132
1     0.047060
2     0.037266
3     0.036784
4     0.044192
        ...   
91    0.000092
92    0.008519
93    0.001261
94    0.015115
95    0.000445
Name: trainMSE_complexmodel, Length: 96, dtype: float64
0     0.047294
1     0.041772
2     0.041337
3     0.062882
4     0.057632
        ...   
91    0.002156
92    0.003412
93    0.002102
94    0.008120
95    0.000911
Name: trainMSE_complexmodel, Length: 96, dtype: float64
0     0.047684
1     0.043333
2     0.050132
3     0.057138
4     0.071915
        ...   
91    0.005232
92    0.002

C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subjdata["experiment"] = experiment_group_temp
C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subjdata["Subject"] = subj
C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

0     0.008546
1     0.008519
2     0.000359
3     0.001202
4     0.001556
        ...   
91    0.039834
92    0.066886
93    0.070888
94    0.052202
95    0.036810
Name: trainMSE_complexmodel, Length: 96, dtype: float64
0     0.039834
1     0.041782
2     0.043029
3     0.055428
4     0.060150
        ...   
91    0.000314
92    0.000359
93    0.001202
94    0.000033
95    0.000288
Name: trainMSE_complexmodel, Length: 96, dtype: float64
0     0.000063
1     0.000092
2     0.007325
3     0.001629
4     0.016104
        ...   
91    0.042992
92    0.040022
93    0.069494
94    0.055428
95    0.064414
Name: trainMSE_complexmodel, Length: 96, dtype: float64
0     0.062228
1     0.047294
2     0.044192
3     0.046603
4     0.042992
        ...   
91    0.002156
92    0.000445
93    0.010216
94    0.000314
95    0.000033
Name: trainMSE_complexmodel, Length: 96, dtype: float64
0     0.002102
1     0.008120
2     0.013202
3     0.005232
4     0.000314
        ...   
91    0.056825
92    0.060

C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subjdata["experiment"] = experiment_group_temp
C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subjdata["Subject"] = subj
C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\748658189.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [5]:
# Extract useful information at the trial level.
df_allsubjs_trials_cols = ["experiment",'mturkID','Subject','first_condition_is50', 'noise_sd', 'sample_size',  'dD', 'stimulus_number',
       'trial_number','response_correctiscomplexmodel','MAE_bestmodel_labels', 'TSE_bestmodel_labels','response_chooseright','response_choosecomplexmodel',
                           "trainMSE_simplemodel","trainMSE_complexmodel",
                           "testMSE_simplemodel","testMSE_complexmodel",
                          "testMSE_Efron_simplemodel","testMSE_Efron_complexmodel",
                          "correction_Efron_simplemodel","correction_Efron_complexmodel", 
                           "trainMAE_simplemodel","trainMAE_complexmodel",
                           "testMAE_simplemodel","testMAE_complexmodel",
                           "testTSE_simplemodel","testTSE_complexmodel",
                           "OLSnoise_est_mean", "ground_truth_labels","MSE_bestmodel_labels_Efron"]
df_alltrials = df_allsubjs_trials_included[df_allsubjs_trials_cols] # Only included subjects

# Assign unique subject index across experiments
df_alltrials["Subject_global"] = pd.factorize(df_alltrials["experiment"].astype(str) + "_" + df_alltrials["Subject"].astype(str))[0]


# Check if subjects are correct on each trial
df_alltrials["response_iscorrect"] = (df_alltrials['response_correctiscomplexmodel']==df_alltrials['response_choosecomplexmodel']).astype(int)
df_alltrials["response_istruegen"] = (df_alltrials["response_choosecomplexmodel"]==df_alltrials["ground_truth_labels"]).astype(int)

# Assign bonus pay
max_bonus = 2.5
min_bonus = 1.9
df_bonus = df_alltrials.groupby(["mturkID"], as_index=False)[["response_iscorrect"]].sum().copy()
df_bonus["bonus"] = (df_bonus["response_iscorrect"] - 0.5) * (max_bonus*2)
df_bonus["bonus"] = min_bonus + (df_bonus["bonus"]-df_bonus["bonus"].min())/(df_bonus["bonus"].max()-df_bonus["bonus"].min()) * (max_bonus-min_bonus)
df_bonus.loc[:,["mturkID", "bonus"]].to_csv("bonus.csv", index=False, header=False)

# So this term more positive -> favors complex model (response label 1)
df_alltrials["trainMSE_modeldiff"] = df_alltrials["trainMSE_simplemodel"] - df_alltrials["trainMSE_complexmodel"]
df_alltrials["correction_Efron_modeldiff"] = df_alltrials["correction_Efron_simplemodel"] - df_alltrials["correction_Efron_complexmodel"]
df_alltrials["noise_var"] = df_alltrials["noise_sd"]**2
df_alltrials["sample_size_inv"] = 1/df_alltrials["sample_size"] 
df_alltrials["trainMAE_modeldiff"] = df_alltrials["trainMAE_simplemodel"] - df_alltrials["trainMAE_complexmodel"]
df_alltrials["testMSE_modeldiff"] = df_alltrials["testMSE_simplemodel"] - df_alltrials["testMSE_complexmodel"]
df_alltrials["testMSE_Efron_modeldiff"] = df_alltrials["testMSE_Efron_simplemodel"] - df_alltrials["testMSE_Efron_complexmodel"]

# Compute biases with respect to each kind of label.
df_alltrials["bias_truegenlabels"] = df_alltrials["response_choosecomplexmodel"] - df_alltrials["ground_truth_labels"]
df_alltrials["bias_testMSElabels"] = df_alltrials["response_choosecomplexmodel"] - df_alltrials["response_correctiscomplexmodel"]
df_alltrials["bias_testMSEEfronlabels"] = df_alltrials["response_choosecomplexmodel"] - df_alltrials["MSE_bestmodel_labels_Efron"]
df_alltrials["bias_testMAElabels"] = df_alltrials["response_choosecomplexmodel"] - df_alltrials["MAE_bestmodel_labels"]
df_alltrials["bias_testTSElabels"] = df_alltrials["response_choosecomplexmodel"] - df_alltrials["TSE_bestmodel_labels"]

# Check if there are NaNs in remaining dataset
nan_counts = df_alltrials.isnull().sum()
print(nan_counts)
# Print the NaN rows
nan_rows_col1 = df_alltrials[df_alltrials["response_choosecomplexmodel"].isna()]
print(nan_rows_col1)

# Remove mturkID information
df_alltrials = df_alltrials.drop('mturkID', axis=1) # axis=1 specifies columns

# #Remove the NaN rows
# df_alltrials[df_alltrials["response_choosecomplexmodel"].isna()] = None

experiment                        0
mturkID                           0
Subject                           0
first_condition_is50              0
noise_sd                          0
sample_size                       0
dD                                0
stimulus_number                   0
trial_number                      0
response_correctiscomplexmodel    0
MAE_bestmodel_labels              0
TSE_bestmodel_labels              0
response_chooseright              0
response_choosecomplexmodel       0
trainMSE_simplemodel              0
trainMSE_complexmodel             0
testMSE_simplemodel               0
testMSE_complexmodel              0
testMSE_Efron_simplemodel         0
testMSE_Efron_complexmodel        0
correction_Efron_simplemodel      0
correction_Efron_complexmodel     0
trainMAE_simplemodel              0
trainMAE_complexmodel             0
testMAE_simplemodel               0
testMAE_complexmodel              0
testTSE_simplemodel               0
testTSE_complexmodel        

C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\422665450.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_alltrials["Subject_global"] = pd.factorize(df_alltrials["experiment"].astype(str) + "_" + df_alltrials["Subject"].astype(str))[0]
C:\Users\liu_s\AppData\Local\Temp\ipykernel_47620\422665450.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_alltrials["response_iscorrect"] = (df_alltrials['response_correctiscomplexmodel']==df_alltrials['response_choosecomplexmodel']).astype(int)
C:\Users\l

In [6]:
# Compute mean bias for each subject in each condition.
groupby_vars = ["experiment","Subject_global","sample_size"]
subjavg_vars = ["response_correctiscomplexmodel","MSE_bestmodel_labels_Efron","response_chooseright", "response_choosecomplexmodel", "bias_truegenlabels", "bias_testMSElabels", "bias_testMSEEfronlabels","bias_testMAElabels", "bias_testTSElabels"]
df_alltrials_biases = df_alltrials.groupby(groupby_vars, as_index=False)[subjavg_vars].mean().copy()
df_alltrials_biases = df_alltrials_biases.rename(columns={"sample_size": "N_train",
                                                          "response_correctiscomplexmodel": "testMSElabels",
                                                          "MSE_bestmodel_labels_Efron": "testMSElabels_Efron",
                                                          "response_chooseright": '%trials choose right side',
                                                          "response_choosecomplexmodel": '%trials choose complex model',
                                                         "bias_truegenlabels": 'bias (true gen)',
                                                         "bias_testMSElabels": 'bias (testMSE)',
                                                         "bias_testMSEEfronlabels": 'bias (testMSE_Efron)',
                                                         "bias_testMAElabels": 'bias (testMAE)',
                                                         "bias_testTSElabels": 'bias (testTSE)',})
df_alltrials_biases["human_Efron_choiceprobdiff"] = df_alltrials_biases["%trials choose complex model"] - df_alltrials_biases["testMSElabels_Efron"]

In [7]:
# Save human datafiles
df_alltrials.to_csv(os.path.join(data_savepath,"df_alltrials.csv"),index=False);
df_alltrials_biases.to_csv(os.path.join(data_savepath,"df_alltrials_biases.csv"),index=False);

# Numpyro NUTS results

In [22]:
# Visualization functions

# ============================================================
# NumPyro MCMC diagnostics WITHOUT ArviZ
# - trace plots (scalars + selected subject-level entries)
# - energy plots + BFMI
# - rank histograms (per-chain)
# - convergence stats: split-Rhat, ESS (Geyer IPS), divergences, accept prob, tree depth
# ============================================================

import numpy as np
import matplotlib.pyplot as plt

# ---------- helpers to fetch samples in [chains, draws, ...] ----------
def get_samples_by_chain(mcmc):
    # returns dict of arrays with leading dims [chains, draws, ...]
    return mcmc.get_samples(group_by_chain=True)

def get_extras_by_chain(mcmc):
    # e.g., {"diverging": [chains, draws], "energy": [chains, draws], ...}
    return mcmc.get_extra_fields(group_by_chain=True)

# ---------- trace plots ----------
def plot_trace_np(samples_by_chain, var_names, subject_idx=slice(0,6), max_cols=3, figsize=(12, 3)):
    """
    Plot traces for scalar params and first few elements of vector params.
    """
    for name in var_names:
        x = samples_by_chain[name]  # shape: [C, T] or [C, T, ...]
        if x.ndim == 2:
            C, T = x.shape
            fig, ax = plt.subplots(1, 1, figsize=figsize)
            for c in range(C):
                ax.plot(np.arange(T), x[c], lw=0.8, alpha=0.8, label=f"chain {c+1}")
            ax.set_title(f"Trace: {name}")
            ax.set_xlabel("draw")
            ax.legend(loc="best", fontsize=8)
            plt.tight_layout(); plt.show()
        else:
            # vector param: plot a grid of selected indices on the last axis
            *_, K = x.shape
            idx = np.arange(K)[subject_idx]
            n_plots = len(idx)
            ncols = min(max_cols, n_plots)
            nrows = int(np.ceil(n_plots / ncols))
            fig, axes = plt.subplots(nrows, ncols, figsize=(figsize[0], figsize[1]*nrows))
            axes = np.array(axes).reshape(-1)
            for k, ax in zip(idx, axes):
                for c in range(x.shape[0]):
                    ax.plot(x[c, :, k], lw=0.8, alpha=0.8, label=f"chain {c+1}")
                ax.set_title(f"{name}[{k}]")
            for ax in axes[n_plots:]:
                ax.axis("off")
            for ax in axes:
                ax.set_xlabel("draw")
            handles, labels = axes[0].get_legend_handles_labels()
            if handles:
                fig.legend(handles, labels, loc="upper right", fontsize=8)
            plt.tight_layout(); plt.show()


            
# ---------- split-Rhat and ESS (Geyer IPS) ----------
def _split_chains(x):
    """x: [chains, draws]; returns [chains*2, draws/2] (drops last draw if odd)."""
    C, T = x.shape
    if T % 2 == 1:
        x = x[:, :-1]
        T -= 1
    first = x[:, :T//2]
    second = x[:, T//2:]
    return np.concatenate([first, second], axis=0)

def rhat_split(x):
    """
    Split-Rhat (Gelman-Rubin). x shape: [chains, draws]
    """
    xs = _split_chains(x)
    m, n = xs.shape  # m = 2*C, n = draws/2
    chain_means = xs.mean(axis=1)
    chain_vars  = xs.var(axis=1, ddof=1)
    B = n * chain_means.var(ddof=1)
    W = chain_vars.mean()
    var_hat = (n - 1)/n * W + B/n
    return np.sqrt(var_hat / W)

def ess_geyer(x):
    """
    Effective sample size using Geyer's initial positive sequence estimator.
    x shape: [chains, draws]
    """
    C, T = x.shape
    # center each chain
    xc = x - x.mean(axis=1, keepdims=True)
    # FFT autocov per chain
    def acov_fft(y):
        # y: [T]
        n = len(y)
        nfft = 1 << (2*n - 1).bit_length()
        fy = np.fft.rfft(y, nfft)
        acf = np.fft.irfft(fy * np.conjugate(fy))[:n]
        acf /= np.arange(n, 0, -1)
        return np.real(acf)
    acov = np.stack([acov_fft(xc[c]) for c in range(C)], axis=0)  # [C, T]
    # normalize to autocorr
    s2 = acov[:, 0]
    rho = acov / s2[:, None]
    # initial positive sequence on paired sums rho_{2k-1}+rho_{2k}
    t = 1
    tau = 1.0
    while t+1 < T:
        pair = rho[:, t] + rho[:, t+1]
        pair_mean = np.mean(pair)
        if pair_mean <= 0:
            break
        tau += 2 * pair_mean
        t += 2
    neff = C * T / tau
    return max(1.0, float(neff))

def summarize_convergence(mcmc, samples_by_chain, core_vars, vector_vars=None, ess_threshold=400, rhat_threshold=1.01):
    """
    Prints split-Rhat and ESS for the listed variables.
    vector_vars: dict like {"theta0_s": slice(0,6), ...} to avoid spamming.
    """
    print("\n=== Convergence summary ===")
    # extras
    extras = get_extras_by_chain(mcmc)
    div = extras.get("diverging")
    acc = extras.get("accept_prob")
    steps = extras.get("num_steps")
    step_size = extras.get("step_size")
    if div is not None:
        div_chain = np.sum(np.asarray(div), axis=1)
        print("Divergences per chain:", div_chain, "| total:", int(div_chain.sum()))
    if acc is not None:
        print("Mean accept prob per chain:", np.round(np.mean(acc, axis=1), 3))
    if steps is not None:
        print("Max tree depth (num_steps) per chain:", np.max(steps, axis=1))
    if step_size is not None:
        print("Step size per chain:", np.squeeze(step_size))

    def report(name, arr):
        # arr: [chains, draws] or [chains, draws, K]
        if arr.ndim == 2:
            rhat = rhat_split(arr)
            ess  = ess_geyer(arr)
            flag = (" !Rhat" if rhat > rhat_threshold else "") + (" !ESS" if ess < ess_threshold else "")
            print(f"{name:>14s}  Rhat={rhat:6.3f}  ESS={ess:8.1f}{flag}")
        else:
            K = arr.shape[-1]
            rhats, esss = [], []
            for k in range(K):
                rhat = rhat_split(arr[..., k])
                ess  = ess_geyer(arr[..., k])
                rhats.append(rhat); esss.append(ess)
            print(f"{name:>14s}  Rhat[min/med/max]={np.min(rhats):.3f}/{np.median(rhats):.3f}/{np.max(rhats):.3f}  "
                  f"ESS[min/med/max]={np.min(esss):.1f}/{np.median(esss):.1f}/{np.max(esss):.1f}  "
                  f"{'!Rhat' if np.max(rhats)>rhat_threshold else ''}{' !ESS' if np.min(esss)<ess_threshold else ''}")

    for v in core_vars:
        report(v, np.asarray(samples_by_chain[v]))

    if vector_vars:
        for v, sl in vector_vars.items():
            x = np.asarray(samples_by_chain[v])[..., sl]
            report(v, x)

            
            

# ============================================================
# KDE per-chain posterior plots (no ArviZ)
# ------------------------------------------------------------
# - Accepts samples with leading dims [chains, draws, ...]
# - Plots each chain's 1D KDE and an optional pooled KDE
# - Handles scalar params ([C,T]) and vector params ([C,T,K])
# - Robust to JAX DeviceArrays (converts to NumPy)
# ============================================================

import numpy as np
import matplotlib.pyplot as plt

try:
    from scipy.stats import gaussian_kde
    _HAS_SCIPY = True
except Exception:
    _HAS_SCIPY = False

def _to_np(x):
    return np.asarray(x)

def _safe_kde_1d(samples, gridsize=256, bw_method="scott", pad=0.05):
    """
    Returns (grid_x, density_y). If KDE isn't available or variance ~ 0, returns (bins_x, hist_y).
    """
    s = _to_np(samples).ravel()
    s = s[np.isfinite(s)]
    if s.size < 2:
        return None, None

    lo, hi = np.quantile(s, [0.001, 0.999])
    if not np.isfinite(lo) or not np.isfinite(hi) or lo == hi:
        return None, None
    span = hi - lo
    lo -= pad * span
    hi += pad * span
    xs = np.linspace(lo, hi, gridsize)

    if _HAS_SCIPY and np.var(s) > 0:
        kde = gaussian_kde(s, bw_method=bw_method)
        ys = kde(xs)
        return xs, ys
    else:
        # Fallback: normalized histogram
        ys, edges = np.histogram(s, bins=min(50, max(10, s.size // 20)), density=True, range=(lo, hi))
        xs_mid = 0.5 * (edges[:-1] + edges[-1:1:-1])  # incorrect; fix below
        xs_mid = 0.5 * (edges[:-1] + edges[1:])
        return xs_mid, ys

def plot_kde_per_chain(
    samples_by_chain,
    var_names,
    subject_idx=slice(0, 6),         # which indices to show for vector params
    pooled=True,                      # overlay pooled KDE
    gridsize=256,
    bw_method="scott",                # or "silverman" or float
    max_cols=3,
    figsize=(12, 3),
    alpha_chain=0.9
):
    """
    Parameters
    ----------
    samples_by_chain : dict
        Output of mcmc.get_samples(group_by_chain=True)
        Shapes: [C,T] for scalars; [C,T,K] for vector params.
    var_names : list of str
        Variable names to plot.
    subject_idx : slice or array-like
        Which indices to plot for vector params (last axis).
    pooled : bool
        If True, plot black dashed KDE for pooled draws.
    """

    # Consistent colors for chains
    # (will repeat if chains > 10)
    chain_colors = plt.cm.tab10.colors

    fig, ax = plt.subplots(len(var_names), 2, figsize=(10, 2*len(var_names)))
    count = 0
    for name in var_names:
        arr = _to_np(samples_by_chain[name])
        if arr.ndim == 2:
            C, T = arr.shape
            # set common x-range using pooled draws (for visual comparability)
            pooled_vals = arr.reshape(C * T)
            lo, hi = np.quantile(pooled_vals[np.isfinite(pooled_vals)], [0.001, 0.999])
            span = hi - lo if hi > lo else 1.0
            lo -= 0.05 * span
            hi += 0.05 * span

            # per-chain KDEs
            for c in range(C):
                xs, ys = _safe_kde_1d(arr[c], gridsize=gridsize, bw_method=bw_method)
                if xs is None:
                    # fallback: rug plot
                    ax[count,0].plot(arr[c], np.zeros_like(arr[c]) + 1e-6, "|", color=chain_colors[c % 10], alpha=0.6)
                else:
                    ax[count,0].plot(xs, ys, color=chain_colors[c % 10], alpha=alpha_chain, lw=1.8, label=f"chain {c+1}")
            # 1) pooled KDE
            if pooled:
                xs_p, ys_p = _safe_kde_1d(pooled_vals, gridsize=gridsize, bw_method=bw_method)
                if xs_p is not None:
                    ax[count,0].plot(xs_p, ys_p, "k--", lw=2.0, alpha=0.9, label="pooled")
            ax[count,0].set_xlim(lo, hi)
            ax[count,0].set_title(f"KDE: {name}")
            ax[count,0].set_xlabel(name)
            ax[count,0].set_ylabel("density")
            ax[0,0].legend(fontsize=8, ncol=min(C+1, 4))
            
            # 2) Per chain traceplots
            for c in range(C):
                ax[count,1].plot(np.arange(T), arr[c], lw=0.8, alpha=0.8, label=f"chain {c+1}")
            ax[count,1].set_title(f"Trace: {name}")
            ax[count,1].set_xlabel("draw")
            #ax[count,1].legend(loc="best", fontsize=8)
            
            count = count + 1

    plt.tight_layout()

In [23]:
# ============================================================
# Hierarchical power-law correction model in NumPyro (NUTS / MCMC)
# Data columns expected in df_alltrials:
#  - Subject_global (int, arbitrary IDs across experiments)
#  - response_choosecomplexmodel (0/1)
#  - OLSnoise_est_mean (float > 0)     # or swap to noise_var if you prefer
#  - sample_size_inv  (float > 0)      # 1/N_train
#  - dD  (Î”D = D_simple - D_complex; in your data Î”D < 0)
#  - trainMSE_modeldiff (Î”trainMSE = trainMSE_simple - trainMSE_complex)
#     NOTE: More positive Î”trainMSE favors choosing the complex model.
# ============================================================
import numpy as np
import pandas as pd
import jax
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
import pickle
import os
import arviz as az

def randomSlopes_NUTS(num_warmup=1000, num_samples=1000, num_chains=2, rng_seed=0):
    
    print("num_warmup={0}, num_samples={1}, num_chains={2}, seed={3}".format(num_warmup, num_samples, num_chains, rng_seed))

    # Set the number of CPU devices for parallel chains (e.g., 4)
    numpyro.set_host_device_count(num_chains) 
    import jax # Import JAX after setting host device count
    num_devices_after_config = jax.local_device_count()
    print("Number of JAX devices after configuration:", str(num_devices_after_config))

    # -----------------------
    # 5) Putting it together
    # -----------------------
    df_alltrials = pd.read_csv(os.path.join("df_alltrials.csv"))
    data = prepare_tensors(df_alltrials)
    mcmc = fit_nuts_np(data, num_warmup=num_warmup, num_samples=num_samples, num_chains=num_chains,
                       rng_seed=rng_seed, chain_method="parallel")
    # Collect the results
    posterior_samples = mcmc.get_samples()
    _ = summarize_posterior_mcmc_np(mcmc)
    pp_mean, pp_draws = posterior_predictive_probs_np(mcmc, data)
    acc = np.mean(((pp_mean > 0.5).astype(np.float32) == np.array(data["y"])))
    print("Posterior predictive mean accuracy:", float(acc))

    file_path = os.path.join(fitspath,"glme_numpyro_"+str(num_warmup)+"_"+str(num_samples)+"_null_seed"+str(rng_seed)+".pickle")
    a = {"data":data, "mcmc":mcmc, "posterior_samples":posterior_samples, "pp_mean":pp_mean, "pp_draws":pp_draws, "acc": acc, "num_warmup": num_warmup, "num_samples": num_samples, "num_chains": num_chains, "rng_seed": rng_seed}
    with open(file_path, 'wb') as file_handle:
        pickle.dump(a, file_handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("Data files saved.")
    mcmc.print_summary()
    

    
# -----------------------
# 0) Prepare tensors (-> JAX DeviceArrays)
# -----------------------
def prepare_tensors(df):
    subj_codes = pd.Categorical(df["Subject_global"])
    subj_idx = jnp.array(subj_codes.codes, dtype=jnp.int32)

    y = 1-jnp.array(df["response_choosecomplexmodel"].values, dtype=jnp.float32)

    # V = jnp.array(df["OLSnoise_est_mean"].values, dtype=jnp.float32)
    V = jnp.array(df["noise_var"].values, dtype=jnp.float32)

    invN   = jnp.array(df["sample_size_inv"].values, dtype=jnp.float32)
    dD_raw = -jnp.array(df["dD"].values, dtype=jnp.float32)
    dTrain = -jnp.array(df["trainMSE_modeldiff"].values, dtype=jnp.float32)

    dTrain_z = dTrain  # keep unscaled to match your Pyro code

    S = int(subj_codes.categories.size)
    N = len(df)

    eps = 1e-12
    return {
        "y": y,
        "V": jnp.clip(V, a_min=eps),
        "invN": jnp.clip(invN, a_min=eps),
        "dD_raw": dD_raw,          # keep signed; do NOT clip here
        "dTrain_z": dTrain_z,
        "subj_idx": subj_idx,
        "S": S,
        "N": N,
    }

# -----------------------
# 1) Model (with signed power for Î”D)
# -----------------------
def powerlaw_logit_model_null(y, V, invN, dD_raw, dTrain_z, subj_idx, S):
    """
    logit P(y=1) = Î¸0_s + Î¸train_s * dTrain_z  - Îº_s * (V^Î±) * [sign(Î”D)*|Î”D|^Î²] * (invN^Î³)

    - Signed power ensures well-defined Î”D^Î² when Î”D<0 and Î²âˆˆâ„.
    - Î±, Î², Î³: population exponents (shared).
    - Î¸0_s, Î¸train_s, Îº_s: subject-level via non-centered parameterization; Îº_s unconstrained.
    """

    # ---- Hyperpriors for subject-level effects ----
    # Intercepts
    mu0    = numpyro.sample("mu0",    dist.Normal(0.0, 10))
    sigma0 = numpyro.sample("sigma0", dist.HalfCauchy(2.5))

    # Slopes on Î”trainMSE
    mu_t    = numpyro.sample("mu_t",    dist.Normal(1.0, 10))
    sigma_t = numpyro.sample("sigma_t", dist.HalfCauchy(2.5))

    with numpyro.plate("subjects", S):
        z0 = numpyro.sample("z0", dist.Normal(0.0, 1.0))
        zt = numpyro.sample("zt", dist.Normal(0.0, 1.0))

        theta0_s     = numpyro.deterministic("theta0_s",     mu0  + sigma0  * z0)
        thetatrain_s = numpyro.deterministic("thetatrain_s", mu_t + sigma_t * zt)
 
    # Gather subject-level params per trial
    theta0_per     = theta0_s[subj_idx]      # [N]
    thetatrain_per = thetatrain_s[subj_idx]  # [N]

    # Linear predictor (minus sign encodes normative direction)
    eta = theta0_per + thetatrain_per * dTrain_z
    with numpyro.plate("data", y.shape[0]):
        numpyro.sample("y_obs", dist.Bernoulli(logits=eta), obs=y)

# -----------------------
# 2) Fit with NUTS / MCMC
# -----------------------
def fit_nuts_np(data, num_warmup=1000, num_samples=1000, num_chains=1,
                rng_seed=0, progress_bar=True, chain_method="sequential",
                target_accept_prob=0.9, max_tree_depth=12):
    """
    chain_method: "sequential" (safe everywhere) or "parallel" (requires XLA multi-processing support).
    """
    numpyro.set_host_device_count(num_chains)
    kernel = NUTS(powerlaw_logit_model_null,
                  target_accept_prob=target_accept_prob,
                  max_tree_depth=max_tree_depth)

    mcmc = MCMC(kernel, num_warmup=num_warmup, num_samples=num_samples,
                num_chains=num_chains, progress_bar=progress_bar,
                chain_method=chain_method)

    rng_key = jax.random.PRNGKey(rng_seed)
    mcmc.run(rng_key,
             data["y"], data["V"], data["invN"], data["dD_raw"],
             data["dTrain_z"], data["subj_idx"], data["S"])
    return mcmc

# -----------------------
# 3) Posterior summaries
# -----------------------
def summarize_posterior_mcmc_np(mcmc):
    s = mcmc.get_samples(group_by_chain=False)  # dict of [draws] or [draws, S]

    def ci(a, q=(0.025, 0.5, 0.975)):
        qs = jnp.quantile(a, jnp.array(q))
        return {"lo": float(qs[0]), "med": float(qs[1]), "hi": float(qs[2])}

    print("\nPosterior (median [95% CI])")
    for name in ["mu0","sigma0","mu_t","sigma_t"]:
        stats = ci(s[name])
        print(f"{name:>8s}: {stats['med']:.3f}  [{stats['lo']:.3f}, {stats['hi']:.3f}]")

    return s

# -----------------------
# 4) Posterior predictive probabilities (per trial)
# -----------------------
def posterior_predictive_probs_np(mcmc, data, thin=None):
    """
    Returns:
      probs_mean: [N] posterior predictive mean P(y=1) per trial
      probs_all:  [draws, N] raw per-draw probabilities (for CIs)
    """
    s = mcmc.get_samples(group_by_chain=False)

    # Optional thinning
    if thin is not None and thin > 1:
        idx = jnp.arange(0, s["alpha"].shape[0], step=thin, dtype=jnp.int32)
        s = {k: v[idx] for k, v in s.items()}

    # Reconstruct subject-level params
    theta0 = s["mu0"][:, None]  + s["sigma0"][:, None]  * s["z0"]   # [draws, S]
    thetat = s["mu_t"][:, None] + s["sigma_t"][:, None] * s["zt"]   # [draws, S]

    V     = data["V"][None, :]        # [1, N]
    invN  = data["invN"][None, :]     # [1, N]
    dD    = data["dD_raw"][None, :]   # [1, N]
    dTr   = data["dTrain_z"][None, :] # [1, N]
    subj  = data["subj_idx"]          # [N]


    # Gather subject-level params per trial (index 2nd axis by subj)
    theta0_per = jnp.take(theta0, subj, axis=1)  # [draws, N]
    thetat_per = jnp.take(thetat, subj, axis=1)  # [draws, N]

    eta = theta0_per + thetat_per * dTr
    probs_all = jax.nn.sigmoid(eta)              # [draws, N]
    probs_mean = jnp.mean(probs_all, axis=0)     # [N]
    return np.array(probs_mean), np.array(probs_all)




#randomSlopes_NUTS(num_warmup=2000, num_samples=2000, num_chains=4, rng_seed=0)

In [24]:
# -----------------------
# 1) Model (with signed power for Î”D)
# -----------------------
def powerlaw_logit_model_coarse(y, V, invN, dD_raw, dTrain_z, subj_idx, S):
    """
    logit P(y=1) = Î¸0_s + Î¸train_s * dTrain_z  - Îº_s * (V^Î±) * [sign(Î”D)*|Î”D|^Î²] * (invN^Î³)

    - Signed power ensures well-defined Î”D^Î² when Î”D<0 and Î²âˆˆâ„.
    - Î±, Î², Î³: population exponents (shared).
    - Î¸0_s, Î¸train_s, Îº_s: subject-level via non-centered parameterization; Îº_s unconstrained.
    """

    # ---- Hyperpriors for subject-level effects ----
    # Intercepts
    mu0    = numpyro.sample("mu0",    dist.Normal(0.0, 10))
    sigma0 = numpyro.sample("sigma0", dist.HalfCauchy(2.5))
    # Slopes on Î”trainMSE
    mu_t    = numpyro.sample("mu_t",    dist.Normal(1.0, 10))
    sigma_t = numpyro.sample("sigma_t", dist.HalfCauchy(2.5))
    # Îº hierarchy (unconstrained, as in your Pyro code)
    mu_k   = numpyro.sample("mu_k",  dist.Normal(1, 10))
    sig_k  = numpyro.sample("sig_k", dist.HalfCauchy(2.5))

    with numpyro.plate("subjects", S):
        z0 = numpyro.sample("z0", dist.Normal(0.0, 1.0))
        zt = numpyro.sample("zt", dist.Normal(0.0, 1.0))
        zk = numpyro.sample("zk", dist.Normal(0.0, 1.0))

        theta0_s     = numpyro.deterministic("theta0_s",     mu0  + sigma0  * z0)
        thetatrain_s = numpyro.deterministic("thetatrain_s", mu_t + sigma_t * zt)
        kappa_s      = numpyro.deterministic("kappa_s",      mu_k + sig_k   * zk)  # could be Â±

    # Power-law correction term
    corr = (V * dD_raw * invN)  # [N]

    # Gather subject-level params per trial
    theta0_per     = theta0_s[subj_idx]      # [N]
    thetatrain_per = thetatrain_s[subj_idx]  # [N]
    kappa_per      = kappa_s[subj_idx]       # [N]

    # Linear predictor (minus sign encodes normative direction)
    eta = theta0_per + thetatrain_per * dTrain_z + 2*kappa_per * corr

    with numpyro.plate("data", y.shape[0]):
        numpyro.sample("y_obs", dist.Bernoulli(logits=eta), obs=y)

# -----------------------
# 2) Fit with NUTS / MCMC
# -----------------------
def fit_nuts_np(data, num_warmup=1000, num_samples=1000, num_chains=1,
                rng_seed=0, progress_bar=True, chain_method="sequential",
                target_accept_prob=0.9, max_tree_depth=12):
    """
    chain_method: "sequential" (safe everywhere) or "parallel" (requires XLA multi-processing support).
    """
    numpyro.set_host_device_count(num_chains)
    kernel = NUTS(powerlaw_logit_model,
                  target_accept_prob=target_accept_prob,
                  max_tree_depth=max_tree_depth)

    mcmc = MCMC(kernel, num_warmup=num_warmup, num_samples=num_samples,
                num_chains=num_chains, progress_bar=progress_bar,
                chain_method=chain_method)

    rng_key = jax.random.PRNGKey(rng_seed)
    mcmc.run(rng_key,
             data["y"], data["V"], data["invN"], data["dD_raw"],
             data["dTrain_z"], data["subj_idx"], data["S"])
    return mcmc

# -----------------------
# 3) Posterior summaries
# -----------------------
def summarize_posterior_mcmc_np(mcmc):
    s = mcmc.get_samples(group_by_chain=False)  # dict of [draws] or [draws, S]

    def ci(a, q=(0.025, 0.5, 0.975)):
        qs = jnp.quantile(a, jnp.array(q))
        return {"lo": float(qs[0]), "med": float(qs[1]), "hi": float(qs[2])}

    print("\nPosterior (median [95% CI])")
    for name in ["mu0","sigma0","mu_t","sigma_t","mu_k","sig_k"]:
        stats = ci(s[name])
        print(f"{name:>8s}: {stats['med']:.3f}  [{stats['lo']:.3f}, {stats['hi']:.3f}]")

    return s



In [25]:

# -----------------------
# 1) Model (with signed power for Î”D)
# -----------------------
def powerlaw_logit_model_fine(y, V, invN, dD_raw, dTrain_z, subj_idx, S):
    """
    logit P(y=1) = Î¸0_s + Î¸train_s * dTrain_z  - Îº_s * (V^Î±) * [sign(Î”D)*|Î”D|^Î²] * (invN^Î³)

    - Signed power ensures well-defined Î”D^Î² when Î”D<0 and Î²âˆˆâ„.
    - Î±, Î², Î³: population exponents (shared).
    - Î¸0_s, Î¸train_s, Îº_s: subject-level via non-centered parameterization; Îº_s unconstrained.
    """

    # ---- Population exponents (normative target = 1) ----
    alpha = numpyro.sample("alpha", dist.Normal(1.0, 1))
    beta  = numpyro.sample("beta",  dist.Normal(1.0, 1))
    gamma = numpyro.sample("gamma", dist.Normal(1.0, 1))

    # ---- Hyperpriors for subject-level effects ----
    # Intercepts
    mu0    = numpyro.sample("mu0",    dist.Normal(0.0, 10))
    sigma0 = numpyro.sample("sigma0", dist.HalfCauchy(2.5))
    # Slopes on Î”trainMSE
    mu_t    = numpyro.sample("mu_t",    dist.Normal(1.0, 10))
    sigma_t = numpyro.sample("sigma_t", dist.HalfCauchy(2.5))
    # Îº hierarchy (unconstrained, as in your Pyro code)
    mu_k   = numpyro.sample("mu_k",  dist.Normal(1, 10))
    sig_k  = numpyro.sample("sig_k", dist.HalfCauchy(2.5))

    with numpyro.plate("subjects", S):
        z0 = numpyro.sample("z0", dist.Normal(0.0, 1.0))
        zt = numpyro.sample("zt", dist.Normal(0.0, 1.0))
        zk = numpyro.sample("zk", dist.Normal(0.0, 1.0))

        theta0_s     = numpyro.deterministic("theta0_s",     mu0  + sigma0  * z0)
        thetatrain_s = numpyro.deterministic("thetatrain_s", mu_t + sigma_t * zt)
        kappa_s      = numpyro.deterministic("kappa_s",      mu_k + sig_k   * zk)  # could be Â±

    ## Signed power for Î”D
    # sign_dD = jnp.sign(dD_raw)
    # abs_dD  = jnp.clip(jnp.abs(dD_raw), a_min=1e-12)

    ## Power-law correction term
    # corr = (V ** alpha) * (sign_dD * (abs_dD ** beta)) * (invN ** gamma)  # [N]
    corr = (V ** alpha) * (dD_raw ** beta) * (invN ** gamma)  # [N]

    # Gather subject-level params per trial
    theta0_per     = theta0_s[subj_idx]      # [N]
    thetatrain_per = thetatrain_s[subj_idx]  # [N]
    kappa_per      = kappa_s[subj_idx]       # [N]

    # Linear predictor (minus sign encodes normative direction)
    eta = theta0_per + thetatrain_per * dTrain_z + 2*kappa_per * corr

    with numpyro.plate("data", y.shape[0]):
        numpyro.sample("y_obs", dist.Bernoulli(logits=eta), obs=y)

# -----------------------
# 2) Fit with NUTS / MCMC
# -----------------------
def fit_nuts_np(data, num_warmup=1000, num_samples=1000, num_chains=1,
                rng_seed=0, progress_bar=True, chain_method="sequential",
                target_accept_prob=0.9, max_tree_depth=12):
    """
    chain_method: "sequential" (safe everywhere) or "parallel" (requires XLA multi-processing support).
    """
    numpyro.set_host_device_count(num_chains)
    kernel = NUTS(powerlaw_logit_model,
                  target_accept_prob=target_accept_prob,
                  max_tree_depth=max_tree_depth)

    mcmc = MCMC(kernel, num_warmup=num_warmup, num_samples=num_samples,
                num_chains=num_chains, progress_bar=progress_bar,
                chain_method=chain_method)

    rng_key = jax.random.PRNGKey(rng_seed)
    mcmc.run(rng_key,
             data["y"], data["V"], data["invN"], data["dD_raw"],
             data["dTrain_z"], data["subj_idx"], data["S"])
    return mcmc

# -----------------------
# 3) Posterior summaries
# -----------------------
def summarize_posterior_mcmc_np(mcmc):
    s = mcmc.get_samples(group_by_chain=False)  # dict of [draws] or [draws, S]

    def ci(a, q=(0.025, 0.5, 0.975)):
        qs = jnp.quantile(a, jnp.array(q))
        return {"lo": float(qs[0]), "med": float(qs[1]), "hi": float(qs[2])}

    print("\nPosterior (median [95% CI])")
    for name in ["alpha","beta","gamma","mu0","sigma0","mu_t","sigma_t","mu_k","sig_k"]:
        stats = ci(s[name])
        print(f"{name:>8s}: {stats['med']:.3f}  [{stats['lo']:.3f}, {stats['hi']:.3f}]")

    for name in ["alpha","beta","gamma"]:
        p_above = float(jnp.mean((s[name] - 1.0) > 0.0))
        print(f"P({name} > 1) = {p_above:.3f} ;  E[{name}] â‰ˆ {float(jnp.mean(s[name])):.3f}")

    return s



In [26]:
# Efron term comparison?
# -----------------------
def posterior_predictive_probs_null(mcmc, data, thin=None):
    """
    Returns:
      probs_mean: [N] posterior predictive mean P(y=1) per trial
      probs_all:  [draws, N] raw per-draw probabilities (for CIs)
    """
    s = mcmc.get_samples(group_by_chain=False)

    # Optional thinning
    if thin is not None and thin > 1:
        idx = jnp.arange(0, s["alpha"].shape[0], step=thin, dtype=jnp.int32)
        s = {k: v[idx] for k, v in s.items()}

    # Reconstruct subject-level params
    theta0 = s["mu0"][:, None]  + s["sigma0"][:, None]  * s["z0"]   # [draws, S]
    thetat = s["mu_t"][:, None] + s["sigma_t"][:, None] * s["zt"]   # [draws, S]

    V     = data["V"][None, :]        # [1, N]
    invN  = data["invN"][None, :]     # [1, N]
    dD    = data["dD_raw"][None, :]   # [1, N]
    dTr   = data["dTrain_z"][None, :] # [1, N]
    subj  = data["subj_idx"]          # [N]

    corr = (V * dD * invN)  # [draws, N]

    # Gather subject-level params per trial (index 2nd axis by subj)
    theta0_per = jnp.take(theta0, subj, axis=1)  # [draws, N]
    thetat_per = jnp.take(thetat, subj, axis=1)  # [draws, N]

    eta = theta0_per + thetat_per * dTr
    probs_all = jax.nn.sigmoid(eta)              # [draws, N]
    probs_mean = jnp.mean(probs_all, axis=0)     # [N]
    return np.array(probs_mean), np.array(probs_all), theta0_per/thetat_per, 2*corr



def posterior_predictive_probs_coarse(mcmc, data, thin=None):
    """
    Returns:
      probs_mean: [N] posterior predictive mean P(y=1) per trial
      probs_all:  [draws, N] raw per-draw probabilities (for CIs)
    """
    s = mcmc.get_samples(group_by_chain=False)

    # Optional thinning
    if thin is not None and thin > 1:
        idx = jnp.arange(0, s["alpha"].shape[0], step=thin, dtype=jnp.int32)
        s = {k: v[idx] for k, v in s.items()}

    # Reconstruct subject-level params
    theta0 = s["mu0"][:, None]  + s["sigma0"][:, None]  * s["z0"]   # [draws, S]
    thetat = s["mu_t"][:, None] + s["sigma_t"][:, None] * s["zt"]   # [draws, S]
    kappa  = s["mu_k"][:, None] + s["sig_k"][:, None]   * s["zk"]   # [draws, S]

    V     = data["V"][None, :]        # [1, N]
    invN  = data["invN"][None, :]     # [1, N]
    dD    = data["dD_raw"][None, :]   # [1, N]
    dTr   = data["dTrain_z"][None, :] # [1, N]
    subj  = data["subj_idx"]          # [N]

    corr = (V * dD * invN)  # [draws, N]

    # Gather subject-level params per trial (index 2nd axis by subj)
    theta0_per = jnp.take(theta0, subj, axis=1)  # [draws, N]
    thetat_per = jnp.take(thetat, subj, axis=1)  # [draws, N]
    kappa_per  = jnp.take(kappa,  subj, axis=1)  # [draws, N]

    eta = theta0_per + thetat_per * dTr + 2*kappa_per * corr
    probs_all = jax.nn.sigmoid(eta)              # [draws, N]
    probs_mean = jnp.mean(probs_all, axis=0)     # [N]
    return np.array(probs_mean), np.array(probs_all), 2*(kappa_per/thetat_per)*corr + theta0_per/thetat_per, 2*corr



def posterior_predictive_probs_fine(mcmc, data, thin=None):
    """
    Returns:
      probs_mean: [N] posterior predictive mean P(y=1) per trial
      probs_all:  [draws, N] raw per-draw probabilities (for CIs)
    """
    s = mcmc.get_samples(group_by_chain=False)

    # Optional thinning
    if thin is not None and thin > 1:
        idx = jnp.arange(0, s["alpha"].shape[0], step=thin, dtype=jnp.int32)
        s = {k: v[idx] for k, v in s.items()}

    # Reconstruct subject-level params
    theta0 = s["mu0"][:, None]  + s["sigma0"][:, None]  * s["z0"]   # [draws, S]
    thetat = s["mu_t"][:, None] + s["sigma_t"][:, None] * s["zt"]   # [draws, S]
    kappa  = s["mu_k"][:, None] + s["sig_k"][:, None]   * s["zk"]   # [draws, S]

    # Broadcast scalars to [draws, N]
    alpha = s["alpha"][:, None]
    beta  = s["beta"][:, None]
    gamma = s["gamma"][:, None]

    V     = data["V"][None, :]        # [1, N]
    invN  = data["invN"][None, :]     # [1, N]
    dD    = data["dD_raw"][None, :]   # [1, N]
    dTr   = data["dTrain_z"][None, :] # [1, N]
    subj  = data["subj_idx"]          # [N]

    # sign_dD = jnp.sign(dD)
    # abs_dD  = jnp.clip(jnp.abs(dD), a_min=1e-12)

    # corr = (V ** alpha) * (sign_dD * (abs_dD ** beta)) * (invN ** gamma)  # [draws, N]
    corr = (V ** alpha) * (dD ** beta) * (invN ** gamma)  # [draws, N]
    corr_norm = V * dD * invN

    # Gather subject-level params per trial (index 2nd axis by subj)
    theta0_per = jnp.take(theta0, subj, axis=1)  # [draws, N]
    thetat_per = jnp.take(thetat, subj, axis=1)  # [draws, N]
    kappa_per  = jnp.take(kappa,  subj, axis=1)  # [draws, N]

    eta = theta0_per + thetat_per * dTr + 2*kappa_per * corr
    probs_all = jax.nn.sigmoid(eta)              # [draws, N]
    probs_mean = jnp.mean(probs_all, axis=0)     # [N]
    
    empirical_V = V ** alpha
    empirical_dD = dD ** beta
    empirical_invN = invN ** gamma
    empirical_correction_norm = (2*kappa_per*corr + theta0_per)/thetat_per + theta0_per/thetat_per
    Efron_correction = 2*corr_norm
    return np.array(probs_mean), np.array(probs_all), empirical_correction_norm, Efron_correction, empirical_V, empirical_dD, empirical_invN


# Load all GLME models and generate simulated data from each

In [27]:
# Generate simulated datasets for each fitted model, Efron normative, and Efron normative with decision noise.

filenames = ["glme_numpyro_2000_2000_fine_seed0.pickle",
            "glme_numpyro_2000_2000_coarse_seed0.pickle",
            "glme_numpyro_2000_2000_null_seed0.pickle"]
model_names = [model_name + str(" Model") for model_name in ["Fine","Coarse","Null"]]

# Compute mean bias for each subject in each condition.
groupby_vars = ["experiment","Subject_global","sample_size"]
subjavg_vars = ["response_chooseright", "response_choosecomplexmodel", "bias_truegenlabels", "bias_testMSElabels", "bias_testMSEEfronlabels"]
mcmc_models = {}
posterior_samples_models = {}
df_allsimtrials_models = {}
df_allsimtrials_biases_models = {}

sim_seed = 0

for model_idx, filename in enumerate(filenames):
    model_name = model_names[model_idx]
    
    # 1) Load HMC file
    with open(os.path.join(fitspath,filename), 'rb') as file:
        loaded_object = pickle.load(file)
    mcmc = loaded_object["mcmc"]
    data = loaded_object["data"]
    pp_mean = loaded_object["pp_mean"]
    posterior_samples = loaded_object["posterior_samples"]
    acc = loaded_object["acc"]
    mcmc_models[model_name] = mcmc
    posterior_samples_models[model_name] = posterior_samples
    #posterior_samples = mcmc.get_samples()
    print(model_name)
    
    simulated_responses_deterministic = (pp_mean > 0.5).astype(np.float32)
    acc = np.mean(simulated_responses_deterministic==(1-df_alltrials["response_correctiscomplexmodel"]))
    acc_Efron = np.mean(simulated_responses_deterministic==(1-df_alltrials["MSE_bestmodel_labels_Efron"]))
    acc_human = np.mean(simulated_responses_deterministic==(1-df_alltrials["response_choosecomplexmodel"]))
    #posterior_samples = mcmc.get_samples()
    print("Posterior predictive mean accuracy to testMSE labels:", float(acc))
    print("Posterior predictive mean accuracy to testMSE_Efron labels:", float(acc_Efron))
    print("Posterior predictive mean accuracy to humans:", float(acc_human))
    print()
    

    # 2) Generate responses either deterministically or randomly.
    np.random.seed(sim_seed)
    #simulated_responses_binary = (pp_mean > 0.5).astype(np.float32)
    simulated_responses_binary = np.random.binomial(n=1, p=pp_mean)
    simulated_responses_prob = pp_mean # Use continuous to avoid binary cutoffs that introduce artificial extremity.
    df_allsimtrials = df_alltrials.copy()
    # df_allsimtrials["response_choosecomplexmodel"] = (1-simulated_responses_prob)
    df_allsimtrials["response_choosecomplexmodel"] = 1-simulated_responses_binary
    
    
    # 3) Compute biases with respect to each kind of label.
    df_allsimtrials["bias_truegenlabels"] = df_allsimtrials["response_choosecomplexmodel"] - df_allsimtrials["ground_truth_labels"]
    df_allsimtrials["bias_testMSElabels"] = df_allsimtrials["response_choosecomplexmodel"] - df_allsimtrials["response_correctiscomplexmodel"]
    df_allsimtrials["bias_testMSEEfronlabels"] = df_allsimtrials["response_choosecomplexmodel"] - df_allsimtrials["MSE_bestmodel_labels_Efron"]
    df_allsimtrials_models[model_name] = df_allsimtrials
    
    # 4) Average over trials for each participant, to plot biases.
    df_allsimtrials_biases = df_allsimtrials.groupby(groupby_vars, as_index=False)[subjavg_vars].mean()
    df_allsimtrials_biases = df_allsimtrials_biases.rename(columns={"sample_size": "N_train",
                                                              "response_chooseright": '%trials choose right side',
                                                              "response_choosecomplexmodel": '%trials choose complex model',
                                                             "bias_truegenlabels": 'bias (true gen)',
                                                             "bias_testMSElabels": 'bias (testMSE)',
                                                             "bias_testMSEEfronlabels": 'bias (testMSE_Efron)',})
    df_allsimtrials_biases_models[model_name] = df_allsimtrials_biases
   
    

### Efron normative labels
model_name = "Efron normative"
df_allsimtrials = df_alltrials.copy()
# df_allsimtrials["response_choosecomplexmodel"] = (1-simulated_responses_prob)
df_allsimtrials["response_choosecomplexmodel"] = df_allsimtrials["MSE_bestmodel_labels_Efron"]
simulated_responses_deterministic = 1-df_allsimtrials["response_choosecomplexmodel"]

acc = np.mean(simulated_responses_deterministic==(1-df_alltrials["response_correctiscomplexmodel"]))
acc_Efron = np.mean(simulated_responses_deterministic==(1-df_alltrials["MSE_bestmodel_labels_Efron"]))
acc_human = np.mean(simulated_responses_deterministic==(1-df_alltrials["response_choosecomplexmodel"]))
#posterior_samples = mcmc.get_samples()
print(model_name)
print("Posterior predictive mean accuracy to testMSE labels:", float(acc))
print("Posterior predictive mean accuracy to testMSE_Efron labels:", float(acc_Efron))
print("Posterior predictive mean accuracy to humans:", float(acc_human))
print()

# 3) Compute biases with respect to each kind of label.
df_allsimtrials["bias_truegenlabels"] = df_allsimtrials["response_choosecomplexmodel"] - df_allsimtrials["ground_truth_labels"]
df_allsimtrials["bias_testMSElabels"] = df_allsimtrials["response_choosecomplexmodel"] - df_allsimtrials["response_correctiscomplexmodel"]
df_allsimtrials["bias_testMSEEfronlabels"] = df_allsimtrials["response_choosecomplexmodel"] - df_allsimtrials["MSE_bestmodel_labels_Efron"]
df_allsimtrials_models[model_name] = df_allsimtrials

# 4) Average over trials for each participant, to plot biases.
df_allsimtrials_biases = df_allsimtrials.groupby(groupby_vars, as_index=False)[subjavg_vars].mean()
df_allsimtrials_biases = df_allsimtrials_biases.rename(columns={"sample_size": "N_train",
                                                          "response_chooseright": '%trials choose right side',
                                                          "response_choosecomplexmodel": '%trials choose complex model',
                                                         "bias_truegenlabels": 'bias (true gen)',
                                                         "bias_testMSElabels": 'bias (testMSE)',
                                                         "bias_testMSEEfronlabels": 'bias (testMSE_Efron)',})
df_allsimtrials_biases_models[model_name] = df_allsimtrials_biases


# Save the simulated data from each fitted model.
file_path = os.path.join(data_savepath,"df_allsimtrials.pickle")
a = {"posterior_samples_models":posterior_samples_models,"mcmc_models":mcmc_models, "data":data, "df_allsimtrials_models":df_allsimtrials_models, "df_allsimtrials_biases_models":df_allsimtrials_biases_models, "seed": sim_seed}
with open(file_path, 'wb') as file_handle:
    pickle.dump(a, file_handle, protocol=pickle.HIGHEST_PROTOCOL)
print("Data files saved.")

Fine Model
Posterior predictive mean accuracy to testMSE labels: 0.5315117252931323
Posterior predictive mean accuracy to testMSE_Efron labels: 0.5682579564489112
Posterior predictive mean accuracy to humans: 0.7504187604690117

Coarse Model
Posterior predictive mean accuracy to testMSE labels: 0.5232935510887772
Posterior predictive mean accuracy to testMSE_Efron labels: 0.5560615577889447
Posterior predictive mean accuracy to humans: 0.7451319095477387

Null Model
Posterior predictive mean accuracy to testMSE labels: 0.5279522613065326
Posterior predictive mean accuracy to testMSE_Efron labels: 0.550251256281407
Posterior predictive mean accuracy to humans: 0.7350293132328308

Efron normative
Posterior predictive mean accuracy to testMSE labels: 0.809463986599665
Posterior predictive mean accuracy to testMSE_Efron labels: 1.0
Posterior predictive mean accuracy to humans: 0.5637562814070352

Data files saved.


In [28]:
# Mean accuracy for experiments
df_temp = df_alltrials.groupby(["experiment","Subject_global"], as_index=False)[["response_istruegen","response_iscorrect"]].mean()
print("Human accuracy with respect to generative labels:")
print(df_temp["response_istruegen"].mean())
print(df_temp["response_istruegen"].std(ddof=1) / np.sqrt(len(np.unique([df_temp["Subject_global"]]))))

print()
print(df_temp.groupby(["experiment"])["response_istruegen"].mean())
print()
print(df_temp.groupby(["experiment"])["response_istruegen"].std(ddof=1)/np.sqrt(df_temp["experiment"].value_counts()))

print("\n-----------\nHuman accuracy with respect to testMSE labels:")
print(df_temp["response_iscorrect"].mean())
print(df_temp["response_iscorrect"].std(ddof=1) / np.sqrt(len(np.unique([df_temp["Subject_global"]]))))
print()
print(df_temp.groupby(["experiment"])["response_iscorrect"].mean())
print()
print(df_temp.groupby(["experiment"])["response_iscorrect"].std(ddof=1)/np.sqrt(df_temp["experiment"].value_counts()))

Human accuracy with respect to generative labels:
0.5376884422110553
0.005905705686361276

experiment
exp1    0.619167
exp2    0.503750
exp3    0.534439
exp4    0.493333
Name: response_istruegen, dtype: float64

experiment
exp1    0.015188
exp2    0.007668
exp3    0.007189
exp4    0.004741
dtype: float64

-----------
Human accuracy with respect to testMSE labels:
0.5388400335008374
0.014693520241667826

experiment
exp1    0.577500
exp2    0.439167
exp3    0.540391
exp4    0.598333
Name: response_iscorrect, dtype: float64

experiment
exp1    0.012676
exp2    0.027310
exp3    0.030983
exp4    0.036587
dtype: float64
